In [1]:
from datetime import datetime
import json
import os
import numpy as np

import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
from keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Dense
import matplotlib.pyplot as plt

from utility.train_data_loader import load_train_data

Using TensorFlow backend.


In [3]:
epochs = 10
batch_size = 256
specialization = "mobile"
gen_test = False

categories_file = open("../data/categories.json", "r")
categories = json.load(categories_file)

all_subcategories = {k.lower(): v for k, v in categories['Mobile'].items()}
all_subcategories.update({k.lower(): v for k, v in categories['Fashion'].items()})
all_subcategories.update({k.lower(): v for k, v in categories['Beauty'].items()})

data_root = "../../"+specialization+"_image/"

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2"

trainData = load_train_data()
testData = pd.read_csv("../data/test.csv")

validation_data_specialized = trainData[trainData['image_path'].str.contains(specialization)][::100]
validation_data_specialized['image_path'] = validation_data_specialized['image_path']. \
    map(lambda x: x.replace(specialization + '_image/', ''))

test_data_specialized = testData[testData['image_path'].str.contains(specialization)]
test_data_specialized['image_path'] = test_data_specialized['image_path'].\
    map(lambda x: x.replace(specialization+'_image/', ''))

inverted_categories_specialized = {k.lower(): v for k, v in categories[specialization.capitalize()].items()}

custom train data used


/Users/jasonlimantoro/.virtualenvs/ndsc_beginner/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
train_data_specialized = trainData[trainData['image_path'].str.contains(specialization)][::]
df = pd.DataFrame()
for k,v in inverted_categories_specialized.items():
    rows = train_data_specialized.loc[train_data_specialized['Category'] == v][:100]
    df = df.append(rows)

train_data_specialized = df

train_data_specialized['image_path'] = train_data_specialized['image_path']. \
    map(lambda x: x.replace(specialization + '_image/', ''))


validation_data_specialized = train_data_specialized[::10]
validation_data_specialized['image_path'] = validation_data_specialized['image_path']. \
    map(lambda x: x.replace(specialization + '_image/', ''))



/Users/jasonlimantoro/.virtualenvs/ndsc_beginner/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [5]:
IMAGE_SIZE = hub.get_expected_image_size(hub.Module(feature_extractor_url))
image_generator = datagen.flow_from_dataframe(train_data_specialized,
                                              directory=os.path.join(data_root),
                                              x_col="image_path",
                                              y_col="item_category",
                                              target_size=IMAGE_SIZE,
                                              color_mode="rgb",
                                              class_mode="categorical",
                                              shuffle=True,
                                              batch_size=64,
                                              )

INFO:tensorflow:Using /var/folders/pq/g3h8qm9x6730w715x51yqq_r0000gp/T/tfhub_modules to cache modules.
Instructions for updating:
Colocations handled automatically by placer.
Found 2648 images belonging to 27 classes.


In [6]:
label_names = sorted(image_generator.class_indices.items(), key=lambda pair:pair[1])
label_names = np.array([key.title() for key, value in label_names])


def feature_extractor(x):
    feature_extractor_module = hub.Module(feature_extractor_url)
    return feature_extractor_module(x)


for image_batch, label_batch in image_generator:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break

Image batch shape:  (64, 224, 224, 3)
Label batch shape:  (64, 27)


In [10]:
model = Sequential()
model.add(Lambda(feature_extractor, input_shape=IMAGE_SIZE+[3], trainable=True))
model.add(Dense(len(inverted_categories_specialized), activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 27)                34587     
Total params: 34,587
Trainable params: 34,587
Non-trainable params: 0
_________________________________________________________________


In [8]:
valid_generator = valid_datagen.flow_from_dataframe(validation_data_specialized,
                                                    directory=os.path.join(data_root),
                                                    x_col="image_path",
                                                    y_col="item_category",
                                                    target_size=IMAGE_SIZE,
                                                    color_mode="rgb",
                                                    class_mode="categorical",
                                                    shuffle=True,
                                                    batch_size=64,
                                                    )


test_generator = test_datagen.flow_from_dataframe(test_data_specialized,
                                                  directory=os.path.join(data_root),
                                                  x_col="image_path",
                                                  y_col=None,
                                                  target_size=IMAGE_SIZE,
                                                  color_mode="rgb",
                                                  class_mode=None,
                                                  shuffle=False,
                                                  batch_size=64,
                                                  )


Found 265 images belonging to 27 classes.
Found 40417 images.


In [12]:
def gen_filename_h5():
    return 'epoch_'+str(epochs) + '_' + datetime.now().strftime("%m_%d_%Y_%H_%M_%S")


def gen_filename_csv():
    return 'epoch_'+str(epochs) + '_' + datetime.now().strftime("%m_%d_%Y_%H_%M_%S")


# Checkpoint auto
filepath = "../checkpoints/"+gen_filename_h5()+"v2.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


steps_per_epoch = image_generator.samples//image_generator.batch_size
valid_steps_per_epoch = valid_generator.samples // valid_generator.batch_size
test_steps_per_epoch = test_generator.samples // test_generator.batch_size

history = model.fit_generator(generator=image_generator,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator,
                              validation_steps=valid_steps_per_epoch,
                              epochs=epochs,
                              callbacks=[checkpointer],
                              )


Epoch 1/10
41/41 [==============================] - 97s 2s/step - loss: 2.5640 - acc: 0.2888 - val_loss: 2.4007 - val_acc: 0.3164

Epoch 00001: val_acc improved from -inf to 0.31641, saving model to ../checkpoints/epoch_10_03_18_2019_03_01_42v2.hdf5
Epoch 2/10
41/41 [==============================] - 90s 2s/step - loss: 2.3001 - acc: 0.3675 - val_loss: 2.1403 - val_acc: 0.4030

Epoch 00002: val_acc improved from 0.31641 to 0.40299, saving model to ../checkpoints/epoch_10_03_18_2019_03_01_42v2.hdf5
Epoch 3/10
41/41 [==============================] - 89s 2s/step - loss: 2.0663 - acc: 0.4333 - val_loss: 2.0617 - val_acc: 0.4229

Epoch 00003: val_acc improved from 0.40299 to 0.42289, saving model to ../checkpoints/epoch_10_03_18_2019_03_01_42v2.hdf5
Epoch 4/10
41/41 [==============================] - 103s 3s/step - loss: 1.8905 - acc: 0.4923 - val_loss: 1.7801 - val_acc: 0.5373

Epoch 00004: val_acc improved from 0.42289 to 0.53731, saving model to ../checkpoints/epoch_10_03_18_2019_03_01_